# OpenAlex Colombia

In [33]:
import requests
import time
import json
from IPython.display import JSON

In [ ]:
co=[]
i=0
count=0
baseurl='https://api.openalex.org/works?filter=authorships.institutions.country_code:co&mailto=restrepo@udea.edu.co&per-page=200'
nc='*'

In [113]:
while nc:
    i+=1
    url=baseurl+f'&cursor={nc}'
    try:
        r=requests.get(url)
        cotmp=r.json().get('results')
    except:
        print('ERROR: bad url')
        break
    if i==1:
        try:
            count=r.json().get('meta').get('count')
        except:
            print('ERROR: bad count')
            break
    kk=[d.pop('abstract_inverted_index') for d in cotmp if d.get('abstract_inverted_index')]
    try:
        nc=r.json().get('meta').get('next_cursor')
    except:
        print('ERROR: bad cursor')
        break
    #progress
    print(f'{i} → {200*i}/{count}',end='\r')
    #Append to dictionary list (WARNING: HIG RAM USAGE)
    for p in cotmp:
        co.append(p)
    #=== backup =====
    f=open('openalexco.jsonl','a')
    for p in cotmp:
        json.dump(p,f)
        f.write("\n")
    f.close()
    #API limits
    time.sleep(0.1)
    #EMERGENGY EXIT
    if i*200>count:
        break

In [114]:
f=open('openalexco.json','w')
json.dump(co,f)
f.close()


In [115]:
len(co)

343057

In [116]:
f=open('openalexco.jsonl','a')
for p in co:
    json.dump(p,f)
    f.write("\n")
f.close()

Medicine & Psychology Colombia

In [137]:
md=[d for d in co if len( {'Medicine','Psychology'}.intersection( 
       [ c.get('display_name') for c in d.get('concepts') if c.get('level')==0 ] 
          ) )>0 ]

In [153]:
len(md)

77352

In [216]:
f=open('openalexco_md.json','w')
json.dump(md,f)
f.close()

Medicine & Psychology Bogotá

In [ ]:
insts=set()
for d in md:
    for a in d.get('authorships'):
        if a.get('institutions'):
            for i in a.get('institutions'):
                if i.get('country_code')=="CO":
                    insts.add(i.get('display_name'))

In [163]:
len(insts)

309

In [173]:
[i for i in insts if i.find('National University of Colombia')>-1]

['National University of Colombia',
 'National University of Colombia at Medellín',
 'National University of Colombia at Manizales']

In [206]:
insts_bg={'El Bosque University',
'National University of Colombia',
'Universidad Antonio Nariño',
'Universidad Externado de Colombia',
'Universidad de Bogotá Jorge Tadeo Lozano',
 'Universidad de Los Andes',
 'Universidad del Rosario',
 'University of La Sabana',
 'University of La Salle'}

In [211]:
mdbg=[]
for d in md:
    l=[ [i.get('display_name') for i in a.get('institutions')] for a in d.get('authorships') if a.get('institutions')]
    if len({item for sublist in l for item in sublist}.intersection(insts_bg))>0:
        mdbg.append(d)

In [212]:
len(mdbg)

18974

In [217]:
f=open('openalexco_mdbg.json','w')
json.dump(md,f)
f.close()